In [4]:
"""
Script to render the asset pricing table
"""

import pandas as pd
import numpy as np
import statsmodels.formula.api as smf
from scipy.stats import ttest_1samp
from regtabletotext import prettify_result
import warnings
warnings.filterwarnings("ignore")

from environ.constants import (
    DEPENDENT_VARIABLES,
    PROCESSED_DATA_PATH,
    STABLE_DICT,
    TABLE_PATH,
)
from environ.process.asset_pricing.assetpricing_functions import reg_fama_macbeth, clean_weekly_panel, univariate_sort, double_sort 


In [5]:
# load factors
ff3 = pd.read_csv(PROCESSED_DATA_PATH/"FF3.csv") 
ltw3 = pd.read_csv(PROCESSED_DATA_PATH/"LTW3.csv")

# load the regression panel dataset
reg_panel = pd.read_pickle(
    PROCESSED_DATA_PATH / "panel_main.pickle.zip", compression="zip"
)

# stable non-stable info dict
stable_nonstable_info = {
    "stablecoin": reg_panel[reg_panel["Token"].isin(STABLE_DICT.keys())],
    "non-stablecoin": reg_panel[~reg_panel["Token"].isin(STABLE_DICT.keys())],
    "all": reg_panel,
}

# How are returns aggregated for each portfolio
ret_agg = 'mean'

DEPENDENT_VARIABLES_bis = ['volume_ultimate_share'] #,'volume_in_share' , 'volume_out_share']
# ,'eigen_centrality_undirected','total_eigen_centrality_undirected','Volume_share']

### Univariate sorting

In [6]:
for dom_variable in DEPENDENT_VARIABLES_bis:
    for is_boom in [-1]:
        n_quantiles = 5
        df_panel = clean_weekly_panel(reg_panel, dom_variable, is_stablecoin = 0, is_boom = is_boom)
        df_panel = pd.merge(df_panel,ff3, on='WeekYear')
        summary_table = univariate_sort(df_panel,dom_variable, n_quantiles, ret_agg = ret_agg)

        # Compute the difference: H minus L
        pivot = df_panel.pivot_table(index='WeekYear', columns='portfolio', values='ret')
        diff_returns = pivot[f'P{n_quantiles}'] - pivot['P1']
        mean_diff = diff_returns.mean() if ret_agg == 'mean' else diff_returns.median()
        std_diff = diff_returns.std(ddof=1)
        t_stat_diff, p_value_diff = ttest_1samp(diff_returns, popmean=0)

        summary_table[f'P{n_quantiles}-P1'] = {
            'Mean': mean_diff,
            't-Stat': t_stat_diff,
            # 'p-value': p_value_diff,
            'StdDev': std_diff,
            'Sharpe':  np.sqrt(365/7) * mean_diff / std_diff
        }
        if is_boom == 1:
            boom_str = " boom"
        elif is_boom == 0:
            boom_str = " bust"
        else:
            boom_str = " alltime"
        summary_table = summary_table.style.set_caption(dom_variable + boom_str)
        display(summary_table)

,P1,P2,P3,P4,P5,P5-P1
Mean,-0.014494,0.040613,0.062108,0.137540,0.152238,0.165757
t-Stat,-1.652736,2.792619,4.301996,7.338394,5.424916,5.427056
StdDev,0.319933,0.511703,0.510425,0.659459,1.011819,0.354873
Sharpe,-0.327124,0.573124,0.878643,1.506046,1.086472,3.372833


In [7]:
test = clean_weekly_panel(reg_panel, 'volume_ultimate_share', is_stablecoin = 0, is_boom = -1)
print(test.Token.nunique())
test.Token.value_counts(ascending=True).head(5)

289


Token
MFT       2
BASE      4
BLXM      4
BMI       4
DUCKER    4
Name: count, dtype: int64

In [8]:
df_panel.groupby('portfolio')['mcap'].median()

portfolio
P1    7.097819e+07
P2    9.669529e+07
P3    1.309419e+08
P4    2.225117e+08
P5    1.466509e+09
Name: mcap, dtype: float64

### Asset pricing factor tests

In [9]:
for dom_variable in DEPENDENT_VARIABLES_bis:
    for is_boom in [-1]:
        n_quantiles = 3
        df_panel = clean_weekly_panel(reg_panel, dom_variable, is_stablecoin = 0, is_boom = is_boom)
        df_panel = pd.merge(df_panel,ff3, on='WeekYear')
        summary_table = double_sort(df_panel,dom_variable, secondary_variable='mcap', n_quantiles=n_quantiles, ret_agg=ret_agg)
        
        if is_boom == 1:
            boom_str = " boom"
        elif is_boom == 0:
            boom_str = " bust"
        else:
            boom_str = "alltime"
        summary_table = summary_table.style.set_caption(dom_variable + boom_str)
        display(summary_table)

primary_portfolio,P1,P2,P3
secondary_portfolio,,,
Q1,0.036122,0.108713,0.314444
Q2,-0.004209,0.031743,0.113267
Q3,0.000965,0.033262,0.037608


In [10]:
import statsmodels.api as sm
for dom_variable in DEPENDENT_VARIABLES_bis:
    for is_boom in [-1]:
        n_quantiles = 3
        df_panel = clean_weekly_panel(reg_panel, dom_variable, is_stablecoin = 0, is_boom = is_boom)
        summary_table = univariate_sort(df_panel,dom_variable, n_quantiles)

        # Compute the difference: H minus L
        pivot = df_panel.pivot_table(index='WeekYear', columns='portfolio', values='ret')
        pivot["CDOM"] = pivot[f"P{n_quantiles}"] - pivot['P1']
        dominance_factor = pivot[["CDOM"]].reset_index()
        assets_panel = clean_weekly_panel(reg_panel, dom_variable, is_stablecoin = 0, is_boom = -1)
        # merge all factors
        factors_data = pd.merge(dominance_factor, ff3, on=["WeekYear"], how="left")
        factors_data = pd.merge(factors_data, ltw3, on=["WeekYear"], how="left")
        # merge factors with returns
        factors_data = pd.merge(factors_data, assets_panel, on=["WeekYear"], how="left")
        factors_data = factors_data.dropna()
        # AP_test_ff3 = smf.ols(formula= 'CDOM ~ MKT + SMB + HML', data=factors_data).fit()
        # prettify_result(AP_test_ff3)
        AP_test_cmkt = smf.ols(formula= 'CDOM ~ CMKT', data=factors_data).fit()
        prettify_result(AP_test_cmkt)
        AP_test_ltw3 = smf.ols(formula= 'CDOM ~ CMKT + CMOM + CSIZE', data=factors_data).fit()
        prettify_result(AP_test_ltw3)
        print("CSIZE factor is long small cap and short large cap")

OLS Model:
CDOM ~ CMKT

Coefficients:
           Estimate  Std. Error  t-Statistic  p-Value
Intercept     0.142       0.003       41.591    0.000
CMKT          0.126       0.037        3.417    0.001

Summary statistics:
- Number of observations: 6,298
- R-squared: 0.002, Adjusted R-squared: 0.002
- F-statistic: 11.677 on 1 and 6296 DF, p-value: 0.001

OLS Model:
CDOM ~ CMKT + CMOM + CSIZE

Coefficients:
           Estimate  Std. Error  t-Statistic  p-Value
Intercept     0.134       0.004       37.264      0.0
CMKT          0.132       0.037        3.565      0.0
CMOM          0.248       0.051        4.903      0.0
CSIZE         0.238       0.052        4.559      0.0

Summary statistics:
- Number of observations: 6,298
- R-squared: 0.009, Adjusted R-squared: 0.009
- F-statistic: 19.339 on 3 and 6294 DF, p-value: 0.000

CSIZE factor is long small cap and short large cap


### FAMA MCBETH 

In [11]:
for dom_variable in DEPENDENT_VARIABLES_bis:
    for is_boom in [-1]:
        n_quantiles = 3
        df_panel = clean_weekly_panel(reg_panel, dom_variable, is_stablecoin = 0, is_boom = is_boom)
        summary_table = univariate_sort(df_panel,dom_variable, n_quantiles)

        # Compute the difference: H minus L
        pivot = df_panel.pivot_table(index='WeekYear', columns='portfolio', values='ret')
        pivot["CDOM"] = pivot[f"P{n_quantiles}"] - pivot['P1']
        dominance_factor = pivot[["CDOM"]].reset_index()
        assets_panel = clean_weekly_panel(reg_panel, dom_variable, is_stablecoin = 0, is_boom = -1)
        # merge all factors
        data_fama_macbeth = pd.merge(dominance_factor, ff3, on=["WeekYear"], how="left")
        data_fama_macbeth = pd.merge(data_fama_macbeth, ltw3, on=["WeekYear"], how="left")
        # merge factors with returns
        data_fama_macbeth = pd.merge(data_fama_macbeth, assets_panel, on=["WeekYear"], how="left")
        data_fama_macbeth = data_fama_macbeth.dropna()

        # run the Fama-Macbeth regression
        data_fama_macbeth['excess_ret'] = data_fama_macbeth['ret'] - data_fama_macbeth['RF']
        fama_macbeth = reg_fama_macbeth(data_fama_macbeth, formula="excess_ret ~ CMKT + CMOM + CSIZE + CDOM")
        if is_boom == 1:
            boom_str = " boom"
        elif is_boom == 0:
            boom_str = " bust"
        else:
            boom_str = " alltime"
        fama_macbeth = fama_macbeth.style.set_caption(dom_variable + boom_str)
        display(fama_macbeth)

,factor,risk_premium,t_stat,t_stat_NW
0,CDOM,1.639000,2.413000,2.166000
1,CMKT,1.064000,5.167000,5.445000
2,CMOM,0.069000,0.649000,0.631000
3,CSIZE,0.363000,3.205000,3.135000
4,Intercept,6.180000,4.243000,3.470000
